Logging something on mlflow to retrieve it

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import mlflow
from azureml.core import Workspace, Dataset
import pandas as pd

In [2]:
ml_client = MLClient.from_config(credential=DefaultAzureCredential())
azureml_mlflow_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
mlflow.set_tracking_uri(azureml_mlflow_uri)

# Connect to the Workspace
ws = Workspace.from_config()

# The default datastore is a blob storage container where datasets are stored
datastore = ws.get_default_datastore()

Found the config file in: /config.json


In [3]:
experiment_name = 'experiment_with_mlflow'
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='', experiment_id='0ed74f1d-0b5a-42a3-b761-2b39bac8b87b', lifecycle_stage='active', name='experiment_with_mlflow', tags={}>

In [4]:
import os
from random import random

mlflow.start_run()
mlflow.log_param("hello_param", "world")
mlflow.log_metric("hello_metric", random())
os.system(f"echo 'hello world' > helloworld.txt")
mlflow.log_artifact("helloworld.txt")

KeyboardInterrupt: 

In [ ]:
mlflow.active_run().info.run_id

<RunInfo: artifact_uri='azureml://experiments/experiment_with_mlflow/runs/f8eeaa57-f975-4743-9e70-5202f9c0ad80/artifacts', end_time=None, experiment_id='0ed74f1d-0b5a-42a3-b761-2b39bac8b87b', lifecycle_stage='active', run_id='f8eeaa57-f975-4743-9e70-5202f9c0ad80', run_uuid='f8eeaa57-f975-4743-9e70-5202f9c0ad80', start_time=1669801474042, status='RUNNING', user_id='0ede9b9c-2b54-4172-a26d-bbeff841eda9'>

In [ ]:
mlflow.end_run()

In [ ]:
mlflow_run.info

NameError: name 'mlflow_run' is not defined

In [ ]:
from azureml.core.run import Run
run=Run.get_context()

In [ ]:
run.log("pippo",1)

Attempted to log scalar metric pippo:
1


Retrieving info from mlflow experiment name

In [ ]:
exp=mlflow.get_experiment_by_name("pipeline-exp")

In [ ]:
import shutil
shutil.rmtree(os.path.join("../src/train/models/", "trained_model"), ignore_errors=True)

In [ ]:
query = "params.model = 'CNN' and params.layers = '10' and metrics.`prediction accuracy` >= 0.945"

In [ ]:
runs=mlflow.search_runs(exp.experiment_id,output_format="list")
last_run=runs[-1]

In [ ]:
runs[-2]

<Run: data=<RunData: metrics={'num_features_dataset': 7.0, 'num_samples_dataset': 889.0}, params={'dataset_ID': '9973e164-6679-46c8-9b4d-6c8ed97accf2',
 'dataset_name': 'Titanic_dataset',
 'dataset_version': '9'}, tags={'azureml.nodeid': '8df270ec',
 'azureml.pipeline': '482e2b83-1c4a-406d-a5c7-65f7c1aa59c6',
 'mlflow.parentRunId': '482e2b83-1c4a-406d-a5c7-65f7c1aa59c6',
 'mlflow.rootRunId': '482e2b83-1c4a-406d-a5c7-65f7c1aa59c6',
 'mlflow.source.name': 'preprocess.py',
 'mlflow.source.type': 'JOB',
 'mlflow.user': 'Giosuè Cotugno'}>, info=<RunInfo: artifact_uri='', end_time=1670261638201, experiment_id='ee673d04-85ca-44af-83d7-aa2a95629c42', lifecycle_stage='active', run_id='f1996f53-6915-4640-86b6-57b91b1ac2ab', run_uuid='f1996f53-6915-4640-86b6-57b91b1ac2ab', start_time=1670261448975, status='FINISHED', user_id='Giosuè Cotugno'>>

In [ ]:
from mlflow.tracking import MlflowClient
# Use MlFlow to retrieve the job that was just completed
client = MlflowClient()
run_id = last_run.info.run_id
print(run_id)
finished_mlflow_run = MlflowClient().get_run(run_id)

metrics = finished_mlflow_run.data.metrics
tags = finished_mlflow_run.data.tags
params = finished_mlflow_run.data.params

print(f'{metrics} - {tags} - {params}')

82a79f85-75da-450f-b97c-152d967ad65a
{'metric_eval': 0.15246636771300448} - {'mlflow.user': 'Giosuè Cotugno', 'mlflow.source.name': 'train.py', 'mlflow.source.type': 'LOCAL', 'mlflow.source.git.commit': '68cf695085f0763f50cef109bdd2d2a0b0acfc43', 'dataset_id': '2ec650af-4f83-4009-baaf-e6d1da5adedc', 'mlflow.rootRunId': '82a79f85-75da-450f-b97c-152d967ad65a', 'mlflow.runName': 'calm_root_tqwly63v'} - {'objective': 'binary:logistic', 'base_score': 'None', 'booster': 'None', 'colsample_bylevel': 'None', 'colsample_bynode': 'None', 'colsample_bytree': 'None', 'gamma': 'None', 'gpu_id': 'None', 'interaction_constraints': 'None', 'learning_rate': 'None', 'max_delta_step': 'None', 'max_depth': '5', 'min_child_weight': 'None', 'monotone_constraints': 'None', 'n_jobs': 'None', 'num_parallel_tree': 'None', 'random_state': '0', 'reg_alpha': 'None', 'reg_lambda': 'None', 'scale_pos_weight': 'None', 'subsample': 'None', 'tree_method': 'None', 'validate_parameters': 'None', 'verbosity': 'None', 'eva

In [ ]:
print(params['dataset_ID'])
dataset_id=params['dataset_ID']
dataset=Dataset.get_by_id(ws,id=dataset_id)

3bdf3597-23e7-4f3f-b24a-0c60686b739b


In [ ]:
df=dataset.to_pandas_dataframe()

In [ ]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.00,1,0,7.2500,S
1,1,1,female,38.00,1,0,71.2833,C
2,1,3,female,26.00,0,0,7.9250,S
3,1,1,female,35.00,1,0,53.1000,S
4,0,3,male,35.00,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
884,0,2,male,27.00,0,0,13.0000,S
885,1,1,female,19.00,0,0,30.0000,S
886,0,3,female,25.14,1,2,23.4500,S
887,1,1,male,26.00,0,0,30.0000,C


# Registering dataset from pandas dataframe

In [ ]:
mlflow

<module 'mlflow' from '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/mlflow/__init__.py'>

In [ ]:

df=pd.read_csv("../data/train/data.csv")
# Register the dataset
ds = Dataset.Tabular.register_pandas_dataframe(
        dataframe=df, 
        name='Titanic_dataset', 
        description='dataset containing the survivors from the Titanic disaster',
        target=datastore
    )

# Display information about the dataset
print(ds.name + " v" + str(ds.version) + ' (ID: ' + ds.id + ")")

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/a1c7c2cc-8dc3-4e23-a612-56b89a529170/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.
Titanic_dataset v1 (ID: 797d5ac0-ecdc-4d19-907e-741af544f910)


In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model as AMLModel

In [ ]:
def get_model(
    model_name: str,
    model_version: int = None,  # If none, return latest model
    tag_name: str = None,
    tag_value: str = None,
    aml_workspace: Workspace = None
) -> AMLModel:
    """
    Retrieves and returns a model from the workspace by its name
    and (optional) tag.

    Parameters:
    aml_workspace (Workspace): aml.core Workspace that the model lives.
    model_name (str): name of the model we are looking for
    (optional) model_version (str): model version. Latest if not provided.
    (optional) tag (str): the tag value & name the model was registered under.

    Return:
    A single aml model from the workspace that matches the name and tag, or
    None.
    """
    if aml_workspace is None:
        print("No workspace defined - using current experiment workspace.")
        aml_workspace = get_current_workspace()

    tags = None
    if tag_name is not None or tag_value is not None:
        # Both a name and value must be specified to use tags.
        if tag_name is None or tag_value is None:
            raise ValueError(
                "model_tag_name and model_tag_value should both be supplied"
                + "or excluded"  # NOQA: E501
            )
        tags = [[tag_name, tag_value]]

    model = None
    if model_version is not None:
        # TODO(tcare): Finding a specific version currently expects exceptions
        # to propagate in the case we can't find the model. This call may
        # result in a WebserviceException that may or may not be due to the
        # model not existing.
        model = AMLModel(
            aml_workspace,
            name=model_name,
            version=model_version,
            tags=tags)
    else:
        models = AMLModel.list(
            aml_workspace, name=model_name, tags=tags, latest=True)
        if len(models) == 1:
            model = models[0]
        elif len(models) > 1:
            raise Exception("Expected only one model")

    return model

In [ ]:
get_model("xgb",aml_workspace=ws)

Model(workspace=Workspace.create(name='mlops-aml-ws', subscription_id='f90533aa-280d-40b9-9949-a7ba0ee9511f', resource_group='mlops-rg'), name=xgb, id=xgb:1, version=1, tags={}, properties={'flavors.python_function': '{\n  "loader_module": "mlflow.xgboost",\n  "python_version": "3.8.5",\n  "data": "model.xgb",\n  "env": "conda.yaml"\n}', 'flavors.xgboost': '{\n  "xgb_version": "1.3.3",\n  "data": "model.xgb",\n  "model_class": "xgboost.sklearn.XGBClassifier",\n  "code": null\n}', 'flavors': 'python_function,xgboost', 'azureml.artifactPrefix': 'ExperimentRun/dcid.b1b5edba-154c-4b71-af74-b64294080b27/xgb', 'model_json': '{"run_id": "b1b5edba-154c-4b71-af74-b64294080b27", "artifact_path": "xgb", "utc_time_created": "2022-11-22 12:03:01.252041", "flavors": {"python_function": {"loader_module": "mlflow.xgboost", "python_version": "3.8.5", "data": "model.xgb", "env": "conda.yaml"}, "xgboost": {"xgb_version": "1.3.3", "data": "model.xgb", "model_class": "xgboost.sklearn.XGBClassifier", "code"

In [ ]:
def register_aml_model(
    model_path,
    model_name,
    model_tags,
    exp,
    run_id,
    dataset_id
):
    try:
        tagsValue = {"area": "titanic_classification",
                     "run_id": run_id,
                     "experiment_name": exp.name}
        tagsValue.update(model_tags)
        

        model = AMLModel.register(
            workspace=exp.workspace,
            model_name=model_name,
            model_path=model_path,
            tags=tagsValue,
            datasets=[('training data',
                       Dataset.get_by_id(exp.workspace, dataset_id))])
        os.chdir("..")
        print(
            "Model registered: {} \nModel Description: {} "
            "\nModel Version: {}".format(
                model.name, model.description, model.version
            )
        )
    except Exception:
        traceback.print_exc(limit=None, file=None, chain=True)
        print("Model registration failed")
        raise

In [ ]:
mlflow.xgboost.load_model("azureml://experiments/Default/runs/b1b5edba-154c-4b71-af74-b64294080b27/artifacts/xgb")

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.3, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=1, num_parallel_tree=1,
              objective='binary:logistic', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='auto',
              use_label_encoder=True, validate_parameters=1, verbosity=None)

# Environment creation

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies
from azureml.core.workspace import Workspace
from azureml.core import ScriptRunConfig, Experiment
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

In [ ]:
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/f90533aa-280d-40b9-9949-a7ba0ee9511f/resourceGroups/mlops-RG/providers/Microsoft.MachineLearningServices/workspaces/mlops-AML-WS',
 'name': 'mlops-AML-WS',
 'identity': {'principal_id': '75cf7e0a-b2ab-41a4-81a6-c6fc9f4e1474',
  'tenant_id': 'e99647dc-1b08-454a-bf8c-699181b389ab',
  'type': 'SystemAssigned'},
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': 'c2d20a6a-b6c3-4509-b119-72420da93433',
 'sdkTelemetryAppInsightsKey': '6d7b2b55-6d18-4126-a3ee-251f78cfa6e5',
 'description': '',
 'friendlyName': 'mlops-AML-WS',
 'containerRegistry': '/subscriptions/f90533aa-280d-40b9-9949-a7ba0ee9511f/resourceGroups/mlops-RG/providers/Microsoft.ContainerRegistry/registries/cotuamlcr',
 'keyVault': '/subscriptions/f90533aa-280d-40b9-9949-a7ba0ee9511f/resourceGroups/mlops-RG/providers/Microsoft.Keyvault/vaults/cotu-AML-KV',
 'applicationInsights': '/subscriptions/f90533aa-280d-40b9-9949-a7ba0ee95

In [ ]:
os.system("ls -l")

total 0
drwxrwxrwx 2 root root 0 Nov 15 09:39 Users


0

In [ ]:
BASE_DIR='/home/azureuser/cloudfiles/code/Users/giosue.cotugno/mlops-titanic'

In [ ]:
myenv = Environment(name="myenv")
conda_dep = CondaDependencies(conda_dependencies_file_path=os.path.join(BASE_DIR,'mlservice/pipeline/ci_dependencies.yml'))
myenv.python.conda_dependencies=conda_dep

In [ ]:
myexp = Experiment(workspace=ws, name = "environment-example")
os.path.join(BASE_DIR,'data/rawdata/train.csv')
data=os.path.join(BASE_DIR,'data/rawdata/train.csv')

In [ ]:
aml_compute_target = "testcotu"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")

found existing compute target.


In [ ]:
os.system("cd Users/giosue.cotugno/mlops-titanic/src/preprocess/")

0

In [ ]:
os.system("ls")

Users


0

In [ ]:
src = ScriptRunConfig(source_directory=".",
                      script="Users/giosue.cotugno/mlops-titanic/src/preprocess/preprocess.py",
                      compute_target=aml_compute,
                      environment=myenv)

run = myexp.submit(config=src)

run.wait_for_completion(show_output=True)

RunId: environment-example_1670252367_408dd8e2
Web View: https://ml.azure.com/runs/environment-example_1670252367_408dd8e2?wsid=/subscriptions/f90533aa-280d-40b9-9949-a7ba0ee9511f/resourcegroups/mlops-rg/workspaces/mlops-aml-ws&tid=e99647dc-1b08-454a-bf8c-699181b389ab

Streaming user_logs/std_log.txt

MSI: Failed to retrieve a token from 'identity-responder-not-enabled/?resource=https://management.core.windows.net/&api-version=2017-09-01' with an error of 'No connection adapters were found for 'identity-responder-not-enabled/?resource=https://management.core.windows.net/&api-version=2017-09-01''.
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CC2TCA2N4 to authenticate.
Interactive authentication successfully completed.
data=None experiment_name=titanic output_data=None
input data: None
Cleaning up all outstanding Run operations, waiting 300.0 secon

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "{\"NonCompliant\":\"Process '/azureml-envs/azureml_22e5fdfe1dd161d6192ee16d8e919d76/bin/python' exited with code 1 and error message 'Execution failed. Process exited with status code 1. Error:   File \\\"/azureml-envs/azureml_22e5fdfe1dd161d6192ee16d8e919d76/lib/python3.7/site-packages/pandas/util/_decorators.py\\\", line 311, in wrapper\\n    return func(*args, **kwargs)\\n  File \\\"/azureml-envs/azureml_22e5fdfe1dd161d6192ee16d8e919d76/lib/python3.7/site-packages/pandas/io/parsers/readers.py\\\", line 586, in read_csv\\n    return _read(filepath_or_buffer, kwds)\\n  File \\\"/azureml-envs/azureml_22e5fdfe1dd161d6192ee16d8e919d76/lib/python3.7/site-packages/pandas/io/parsers/readers.py\\\", line 482, in _read\\n    parser = TextFileReader(filepath_or_buffer, **kwds)\\n  File \\\"/azureml-envs/azureml_22e5fdfe1dd161d6192ee16d8e919d76/lib/pytho

In [ ]:
metric_eval = ["mse","precision","recall","fscore"]
metrics,tags,params=get_train_exp()
print(metrics)
allow_run_cancel="true"
properties=[]
for elm in metric_eval:
    properties.append(metrics[elm])